In [73]:
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms

In [74]:
# # Fashion MNIST dataset: too many features: 28x28 for a random tree
# path = 'C:/Users/jeanb/Documents/Python Scripts/DiveIntoDL/data'

# class FashionMNIST(nn.Module):
#     def __init__(self, batch_size = 64, resize=(28,28)):
#         super().__init__()
#         trans = transforms.ToTensor()
#         self.train = torchvision.datasets.FashionMNIST(root=path,
#                                                        train=True,
#                                                        transform=trans,
#                                                        download=True)
#         self.val = torchvision.datasets.FashionMNIST(root=path,
#                                                        train=False,
#                                                        transform=trans,
#                                                        download=True)
        
#     def text_labels(self, indices):
#         labels = ['t-shirt', 'trouser', 'pullover', 'dress', 'coat',
#                    'sandal', 'shirt', 'sneaker', 'bag', 'ankle boot']
#         return [labels[int(i)] for i in indices]
    

# data = FashionMNIST(resize=(32,32))
# data.train

In [75]:
# data.train[0][0].shape # image: 1, 28, 28
# data.train[0][1]       # label: 9
# len(data.train) # 60,000
# count_per_label = {}
# for i in range(len(data.train)):
#     label = data.train[i][1]
#     if label in count_per_label:
#         count_per_label[label] += 1
#     else: # create new label
#         count_per_label[label] = 1
# print(count_per_label) # 6000 items for each label from 0 to 9

# X_train = torch.cat([data.train[i][0] for i in range(len(data.train))], dim=0)
# y_train = torch.cat([torch.Tensor([data.train[i][1]]) for i in range(len(data.train))], dim=0)
# print("Training:", X_train.shape, y_train.shape)
# X_val = torch.cat([data.val[i][0] for i in range(len(data.val))], dim=0)
# y_val = torch.cat([torch.Tensor([data.val[i][1]]) for i in range(len(data.val))], dim=0)
# print("Validation:", X_val.shape, y_val.shape)

In [76]:
import pandas as pd

path = 'C:/Users/jeanb/Documents/Python Scripts/DiveIntoDL/data/'
data = pd.read_csv(path + 'iris.csv')
data.shape # 150, 5
data.keys() # 'sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'species'
features = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']
label = 'species'
train_idx = list(range(0, 40)) + list(range(50, 90)) + list(range(100, 140))
val_idx = list(range(40, 50)) + list(range(90, 100)) + list(range(140, 150))
type(data['sepal_length'][val_idx])

# from text labels to id labels
label_idx = {}
idx = 0
for i in range(data.shape[0]):
    label_name = data[label][i]
    if label_name not in label_idx:
        label_idx[label_name] = idx
        idx += 1
print(label_idx)

{'setosa': 0, 'versicolor': 1, 'virginica': 2}


In [78]:
X_train = torch.zeros((len(train_idx), len(features)))
y_train = torch.zeros(len(train_idx))

X_val = torch.zeros((len(val_idx), len(features)))
y_val = torch.zeros(len(val_idx))

for i, k in enumerate(features):
    X_train[:, i] = torch.tensor(data[k][train_idx].values, dtype=torch.float32)
    X_val[:, i] = torch.tensor(data[k][val_idx].values, dtype=torch.float32)


for i, j in enumerate(train_idx):
    idx = label_idx[ data[label][j] ]
    y_train[i] = torch.tensor(idx)

for i, j in enumerate(val_idx):
    idx = label_idx[ data[label][j] ]
    y_val[i] = torch.tensor(idx)


In [79]:
print("Training:", X_train.shape, y_train.shape)
print("Evaluation:", X_val.shape, y_val.shape)


Training: torch.Size([120, 4]) torch.Size([120])
Evaluation: torch.Size([30, 4]) torch.Size([30])


In [84]:
class DecisionTreeNode:
    def __init__(self, depth=0, max_depth=None):
        self.depth = depth
        self.max_depth = max_depth
        self.feature_index = None
        self.threshold = None
        self.left = None  # Left child
        self.right = None  # Right child
        self.is_leaf = False
        self.predicted_class = None

    def fit(self, X, y):
        # If all labels are the same, make a leaf node
        if y.unique().numel() == 1:
            self.is_leaf = True
            self.predicted_class = y[0].item()
            print(f"Depth {self.depth} class {self.predicted_class} since y={y}")
            return

        # Check if maximum depth is reached
        if self.max_depth is not None and self.depth >= self.max_depth:
            self.is_leaf = True
            self.predicted_class = y.mode()[0].item() # majority vote among the y
            return

        # Find the best split
        best_gini = float('inf')
        num_features = X.shape[1]
        for feature in range(num_features):
            thresholds = torch.unique(X[:, feature])
            for threshold in thresholds:
                left_mask = X[:, feature] <= threshold
                right_mask = X[:, feature] > threshold
                y_left = y[left_mask]
                y_right = y[right_mask]
                if y_left.numel() == 0 or y_right.numel() == 0:
                    continue # skip if this does not split the dataset
                gini = self._gini(y_left, y_right)
                if gini < best_gini:
                    best_gini = gini
                    self.feature_index = feature
                    self.threshold = threshold.item()
                    best_left_mask = left_mask
                    best_right_mask = right_mask

        # If no valid split is found, make a leaf node
        if self.feature_index is None:
            self.is_leaf = True
            self.predicted_class = y.mode()[0].item()
            return

        # Recursively build the left and right subtrees
        self.left = DecisionTreeNode(
            depth=self.depth + 1, max_depth=self.max_depth)
        self.right = DecisionTreeNode(
            depth=self.depth + 1, max_depth=self.max_depth)
        self.left.fit(X[best_left_mask], y[best_left_mask])
        self.right.fit(X[best_right_mask], y[best_right_mask])

    def predict(self, X):
        if self.is_leaf:
            return torch.full((X.shape[0],), self.predicted_class, dtype=torch.long)
        else:
            left_mask = X[:, self.feature_index] <= self.threshold
            right_mask = X[:, self.feature_index] > self.threshold
            y_pred = torch.empty(X.shape[0], dtype=torch.long)
            y_pred[left_mask] = self.left.predict(X[left_mask])
            y_pred[right_mask] = self.right.predict(X[right_mask])
            return y_pred

    def _gini(self, y_left, y_right):
        # Compute Gini impurity
        def gini_impurity(group):
            if group.numel() == 0:
                return 0.0
            classes, counts = torch.unique(group, return_counts=True)
            probabilities = counts.float() / counts.sum()
            return 1.0 - torch.sum(probabilities ** 2).item()

        total_samples = y_left.numel() + y_right.numel()
        gini_left = gini_impurity(y_left)
        gini_right = gini_impurity(y_right)
        weighted_gini = (
            (y_left.numel() / total_samples) * gini_left
            + (y_right.numel() / total_samples) * gini_right
        )
        return weighted_gini
    
    
# Initialize the Decision Tree with a maximum depth
max_depth = 4  # You can adjust this value, but since the dataset is small it could overfit easily
decision_tree = DecisionTreeNode(max_depth=max_depth)

# Train the Decision Tree
decision_tree.fit(X_train, y_train)

print(f"Decision Tree trained with max depth {max_depth}")

Depth 1 class 0.0 since y=tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
Depth 4 class 1.0 since y=tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1.])
Depth 4 class 2.0 since y=tensor([2.])
Depth 4 class 2.0 since y=tensor([2., 2., 2.])
Depth 4 class 1.0 since y=tensor([1.])
Depth 4 class 2.0 since y=tensor([2., 2.])
Depth 3 class 2.0 since y=tensor([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
        2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])
Decision Tree trained with max depth 4


In [85]:
# Make predictions on the training set
predictions = decision_tree.predict(X_train)

# Calculate accuracy
accuracy = (predictions == y_train).float().mean()
print(f"Training Accuracy: {accuracy.item() * 100:.2f}%")

Training Accuracy: 99.17%


In [86]:
# Make predictions on the test set
predictions = decision_tree.predict(X_val)

# Calculate accuracy
accuracy = (predictions == y_val).float().mean()
print(f"Test Accuracy: {accuracy.item() * 100:.2f}%")

Test Accuracy: 100.00%


In [87]:
root = decision_tree
# count nodes
def dfs(node, nb=0, nb_leaves=0):
    print(node.depth, node.feature_index, node.predicted_class)
    if node.is_leaf:
        return 1, nb_leaves + 1
    left_nodes, left_leaves = dfs(node.left)
    right_nodes, right_leaves = dfs(node.right)
    return nb + left_nodes + right_nodes + 1, nb_leaves + left_leaves + right_leaves

nb_nodes, nb_leaves = dfs(root, 0, 0)
print(nb_nodes, "nodes in the tree and", nb_leaves, "leaves")
    

0 2 None
1 None 0.0
1 3 None
2 2 None
3 3 None
4 None 1.0
4 None 2.0
3 3 None
4 None 2.0
4 None 1.0
2 2 None
3 0 None
4 None 1.0
4 None 2.0
3 None 2.0
15 nodes in the tree and 8 leaves
